In [229]:
import numpy as np
import os
import copy

import open3d as o3d
from open3d import JVisualizer

In [230]:
class Mesh:
    def __init__(self, filepath):
        os.path.isfile(filepath)
        self.mesh_o3d = o3d.io.read_triangle_mesh(filepath)
        self.print_mesh()

    def visualize(self):
        self.mesh_o3d.compute_vertex_normals()
        vis = o3d.visualization.Visualizer()
        vis.create_window()
        vis.get_render_option().mesh_show_back_face = True
        self.add_to_vis(vis)
        mesh_frame = o3d.geometry.create_mesh_coordinate_frame(size=4,
                                                               origin=[0, 0, 0])
        vis.add_geometry(mesh_frame)
        vis.run()
        vis.destroy_window()

    def add_to_vis(self, vis):
        """ Add the mesh to the visualization `vis` object """
        self.mesh_o3d.compute_vertex_normals()
        vis.add_geometry(self.mesh_o3d)

    def print_mesh(self):
        print("Testing mesh in open3d ...")
        print(self.mesh_o3d)
        print(np.asarray(self.mesh_o3d.vertices))
        print(np.asarray(self.mesh_o3d.triangles))
        print("")
    
    def transform(self, T):
        self.mesh_o3d.transform(T)

In [231]:
def quaternion_to_matrix(q):
    # q: numpy array: [x, y, z, w]
    x = q[0]
    y = q[1]
    z = q[2]
    w = q[3]
    Nq = w*w + x*x + y*y + z*z
    if Nq < np.finfo(np.float).eps:
        return np.eye(3)
    s = 2.0/Nq
    X = x*s
    Y = y*s
    Z = z*s
    wX = w*X; wY = w*Y; wZ = w*Z
    xX = x*X; xY = x*Y; xZ = x*Z
    yY = y*Y; yZ = y*Z; zZ = z*Z
    return np.array(
           [[ 1.0-(yY+zZ), xY-wZ, xZ+wY ],
            [ xY+wZ, 1.0-(xX+zZ), yZ-wX ],
            [ xZ-wY, yZ+wX, 1.0-(xX+yY) ]])

In [232]:
# FILL PATHS BELOW
gt_mesh_path = "/home/yunchang/Desktop/dcist_gt_mesh/city/city_gt.ply"
est_mesh_path = "/home/yunchang/Desktop/logs-dcist-city/opt_merged.ply"

In [233]:
# FILL IN 
robot_0_position_0 = np.array([10.461223, 6.989249, 0.112946])
robot_0_quaternion_0 = np.array([0, 0, -0.01, 0.99])

In [234]:
print("Loading Ground-truth mesh...")
gt_mesh_original = Mesh(gt_mesh_path)
print("Loading Estimated mesh...")
est_mesh_original = Mesh(est_mesh_path)

Loading Ground-truth mesh...
Testing mesh in open3d ...
geometry::TriangleMesh with 29678634 points and 35058248 triangles.
[[-48.725    59.6745    6.875  ]
 [-48.734    59.675     6.875  ]
 [-48.725    59.675     6.84616]
 ...
 [-48.825    59.6819    7.125  ]
 [-48.775    59.6859    7.125  ]
 [-48.825    59.6842    7.175  ]]
[[       0        1        2]
 [       3        4        5]
 [       6        7        8]
 ...
 [26610404 26610378 29678548]
 [26610430 29678569 29678590]
 [26610430 26610404 29678569]]

Loading Estimated mesh...
Testing mesh in open3d ...
geometry::TriangleMesh with 8009335 points and 13404390 triangles.
[[ 30.4187  -16.5139   10.5214 ]
 [ 30.4893  -16.4418   10.5248 ]
 [ 30.4893  -16.5166   10.5041 ]
 ...
 [  3.86911   5.13399  13.1148 ]
 [  3.79295   5.19404  13.141  ]
 [  3.74991   5.21894  13.1532 ]]
[[      0       1       2]
 [      2       1       3]
 [      4       5       6]
 ...
 [8009268 8009309 8009286]
 [8009334 8009309 8009268]
 [7941730 8009292 800

In [235]:
# Transform Meshes to same frame of reference
gt_mesh = copy.deepcopy(gt_mesh_original)
est_mesh = copy.deepcopy(est_mesh_original)

In [236]:
# Transform gt mesh to est mesh 
w_T_gtPose0 = np.eye(4)
w_T_gtPose0[:3, :3] = quaternion_to_matrix(robot_0_quaternion_0)
w_T_gtPose0[:3, 3] = robot_0_position_0

w_T_estPose0 = np.eye(4) # first pose of first robot after dpgo is identity 

est_T_gt = np.dot(np.linalg.inv(w_T_gtPose0), w_T_estPose0)
gt_mesh.transform(est_T_gt)

In [237]:
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.get_render_option().mesh_show_back_face = True
vis.add_geometry(est_mesh.mesh_o3d)
vis.add_geometry(gt_mesh.mesh_o3d)
vis.add_geometry(o3d.geometry.create_mesh_coordinate_frame(size=4))
vis.run()
vis.destroy_window()

In [238]:
NUMBER_OF_SAMPLES=1000000
gt_pcl = o3d.geometry.sample_points_uniformly(gt_mesh.mesh_o3d, NUMBER_OF_SAMPLES)
# Don't sample estimated mesh, just pick vertices, otw you'll be mixing colors...
# est_pcl = o3d.geometry.sample_points_uniformly(est_mesh.mesh_o3d, NUMBER_OF_SAMPLES)
est_pcl = o3d.io.read_point_cloud(est_mesh_path)

In [239]:
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.get_render_option().mesh_show_back_face = True
vis.add_geometry(gt_pcl)
vis.add_geometry(est_pcl)
vis.run()
vis.destroy_window()

In [240]:
# ICP
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
def draw_correspondences(source, target, correspondences):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    #source_temp.paint_uniform_color([1, 0.706, 0])
    #target_temp.paint_uniform_color([0, 0.651, 0.929])
    o3d.visualization.draw_geometries([source_temp, #target_temp, 
                                       correspondences])
    

In [241]:
# ICP params
ICP_THRESHOLD = 1.0
trans_init = np.asarray([[1.0, 0.0, 0.0, 0.0],
                         [0.0, 1.0, 0.0, 0.0],
                         [0.0, 0.0, 1.0, 0.0],
                         [0.0, 0.0, 0.0, 1.0]])

In [242]:
# Visualize initial registration problem
draw_registration_result(est_pcl, gt_pcl, trans_init)

In [243]:
# Evaluate current fit between pointclouds
evaluation = o3d.registration.evaluate_registration(est_pcl, gt_pcl, ICP_THRESHOLD, trans_init)

In [244]:
print("Initial registration")
print(evaluation)

Initial registration
registration::RegistrationResult with fitness = 0.409029, inlier_rmse = 0.589045, and correspondence_set size of 3276047
Access transformation to get result.


In [245]:
print("Apply point-to-point ICP")
reg_p2p = o3d.registration.registration_icp(
    est_pcl, gt_pcl, ICP_THRESHOLD, trans_init,
    o3d.registration.TransformationEstimationPointToPoint(),
    o3d.registration.ICPConvergenceCriteria(max_iteration = 2000))
correspondences = reg_p2p.correspondence_set

Apply point-to-point ICP


In [246]:
print(reg_p2p)
print("")

print("Transformation is:")
print(reg_p2p.transformation)
print("")

print("Correspondence Set:")
print(reg_p2p.correspondence_set)
print("")

registration::RegistrationResult with fitness = 0.592956, inlier_rmse = 0.513472, and correspondence_set size of 4749181
Access transformation to get result.

Transformation is:
[[ 0.99860839  0.03681391 -0.03776261  1.90568641]
 [-0.03746641  0.99915802 -0.01671903 -2.07746441]
 [ 0.03711532  0.01811059  0.99914687 -0.29410877]
 [ 0.          0.          0.          1.        ]]

Correspondence Set:
std::vector<Eigen::Vector2i> with 4749181 elements.
Use numpy.asarray() to access data.



In [247]:
# Draw Registration Result
draw_registration_result(est_pcl, gt_pcl, reg_p2p.transformation)

In [248]:
# Draw Only Correspondences
c2c_lines = o3d.geometry.create_line_set_from_point_cloud_correspondences(est_pcl, gt_pcl, correspondences)
o3d.visualization.draw_geometries([c2c_lines])

In [249]:
# Draw PointClouds and Correspondences
draw_correspondences(est_pcl, gt_pcl, c2c_lines)

In [250]:
def calc_corresp(est_pcl, gt_pcl, correspondences):
    total_negative_matches = 0
    total_positive_matches = 0
    total_correspondences = len(correspondences)
    for correspondence in correspondences:
        if np.allclose(est_pcl.colors[correspondence[0]],
                       gt_pcl.colors[correspondence[1]]):
            total_positive_matches += 1
        else:
            total_negative_matches += 1

    print("Positive color matches: ",total_positive_matches)
    print("Negative color matches: ", total_negative_matches)
    print("Total correspondences: ", total_correspondences)
    assert(total_correspondences == total_negative_matches + total_positive_matches)
    print ("Positive: {}  % ".format(total_positive_matches / total_correspondences * 100))
    print ("Negative: {}  % ".format(total_negative_matches / total_correspondences * 100))

In [251]:
calc_corresp(est_pcl, gt_pcl, correspondences)

Positive color matches:  3748508
Negative color matches:  1000673
Total correspondences:  4749181
Positive: 78.92956701376511  % 
Negative: 21.07043298623489  % 
